# Creating Redshift cluster with IAC
Use this notebook to create and later delete our cluster along with its corresponding IAM role.

In [1]:
#loading libraries for IAC 
import pandas as pd
import boto3
import json
import configparser

## Load AWS, DWH and Cluster Params from dwh.cfg file

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

# Creates clients for IAM, EC2, S3 and Redshift

In [3]:
# We use the us-west-2 region where all data is located.
ec2 = boto3.resource('ec2',
                       region_name = "us-west-2",
                        aws_access_key_id = KEY,
                        aws_secret_access_key = SECRET)

s3 = boto3.resource('s3',
                       region_name = "us-west-2",
                        aws_access_key_id = KEY,
                        aws_secret_access_key = SECRET)

iam = boto3.client('iam',
                       region_name = "us-west-2",
                        aws_access_key_id = KEY,
                        aws_secret_access_key = SECRET)

redshift = boto3.client('redshift',
                       region_name = "us-west-2",
                        aws_access_key_id = KEY,
                        aws_secret_access_key = SECRET)

# Creates IAM Role that allows Redshift to connect with S3 with ReadOnly access.

In [4]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
    Path = '/',
    RoleName=DWH_IAM_ROLE_NAME,
    Description = "Allows Redshift clusters to call AWS services on your behalf.",
    AssumeRolePolicyDocument=json.dumps(
    {'Statement': [{'Action':'sts:AssumeRole',
     'Effect': 'Allow',
     'Principal': {'Service': 'redshift.amazonaws.com'}}],
      'Version': '2012-10-17'})
    )
    
except Exception as e:
    print(e)

1.1 Creating a new IAM Role


In [5]:
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                          PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                          )['ResponseMetadata']['HTTPStatusCode']
print("1.2 Attaching Policy")

print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::133345115698:role/dwhRole


# Creates a Redshift cluster with our specified IAM role.

In [6]:
try:
    response = redshift.create_cluster(        
        # TODO: add parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        
        # TODO: add parameter for role (to allow s3 access)
        IamRoles=[roleArn]
         
    )
except Exception as e:
    print(e)

In [11]:
# We use the following function to display the status of our Redshift cluster to know when is available or deleted.
# Run as many times until status is marked as available.
def prettyRedshiftProps(props):
    """
    Displays desired columns within cluster description dictionary in simple pandas dataframe.
    
    Keyword arguments:
    props -- dictionary of Redshift cluster description.
    
    """
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.ciuetmmzdpq2.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-036b34160147dc191
7,NumberOfNodes,4


# Use Endpoint and ARN strings from cluster to save on config file.

In [12]:
# We extract the host and ARN role for our cluster that will allows us to connect and continue with our ETL process.
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.ciuetmmzdpq2.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::133345115698:role/dwhRole


In [13]:
#### TCP port to authorize ingress and to port 5439 has been created already ec2.SecurityGroup(id='sg-05d35cc8dfbfbfe33')
#### ELSE unquote below ####

"""
# Opens a TCP port to access Redshift cluster
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)
"""

In [5]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.ciuetmmzdpq2.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2021, 10, 3, 1, 56, 9, 74000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-05d35cc8dfbfbfe33',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-036b34160147dc191',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'tue:10:30-tue:11:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting

In [9]:
# Use to know when cluster status marks as deleted.
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwhcluster not found.

In [11]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

NoSuchEntityException: An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: The role with name dwhRole cannot be found.